## Solving Normal Form Games

In [1]:
from typing import List, Tuple
from copy import deepcopy

The Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> Tuple:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

<a id="strong_compare"></a>
### strong_compare

`strong_compare` compares two lists to see if all values of one list are greater than the values in the other list. Also removes values that have already been deemed as dominated from being compared. **Used by**: [player1](#player1), [player2](#player2)

* **list1** List[int]: the list of integer values
* **list2** List[int]: the list of integer values
* **remove** List[int]: the list of data points to remove before comparing the lists

**return**: int: integer representation of the result. 1 for list1 strongly dominating list2. 2 for list2 strongly dominating list1. 0 for no list dominates the other.

In [3]:
def strong_compare(list1, list2, remove) -> int:
    _list1 = deepcopy(list1)
    _list2 = deepcopy(list2)

    for i in sorted(remove, reverse=True):
        _list1.pop(i)
        _list2.pop(i)
    if all(map(lambda x, y: x > y, _list1, _list2)):
        return 1
    if all(map(lambda x, y: x < y, _list1, _list2)):
        return 2
    return 0

In [4]:
l1 = [12, 20, 28]
l2 = [15, 15, 25]
assert strong_compare(l1, l2, []) == 0
l1 = [12, 20, 28]
l2 = [10, 10, 25]
assert strong_compare(l1, l2, []) == 1
l1 = [12, 20, 28]
l2 = [15, 25, 30]
assert strong_compare(l1, l2, []) == 2

<a id="weak_compare"></a>
### weak_compare

`weak_compare` compares two lists to see if any values of one list are greater than the values in the other list with the rest being at least equal. Also removes values that have already been deemed as dominated from being compared.  **Used by**: [player1](#player1), [player2](#player2)

* **list1** List[int]: the list of integer values
* **list2** List[int]: the list of integer values
* **remove** List[int]: the list of data points to remove before comparing the lists

**return**: int: integer representation of the result. 1 for list1 weakly dominating list2. 2 for list2 weakly dominating list1. 0 for no list dominates the other.

In [5]:
def weak_compare(list1, list2, remove) -> int:
    _list1 = deepcopy(list1)
    _list2 = deepcopy(list2)

    for i in sorted(remove, reverse=True):
        _list1.pop(i)
        _list2.pop(i)
    if all(map(lambda x, y: x >= y, _list1, _list2)):
        if any(map(lambda x, y: x > y, _list1, _list2)):
            return 1
    if all(map(lambda x, y: x <= y, _list1, _list2)):
        if any(map(lambda x, y: x < y, _list1, _list2)):
            return 2
    return 0

In [6]:
l1 = (12, 15, 28)
l2 = (15, 15, 25)
assert weak_compare(l1, l2, []) == 0
l1 = (15, 15, 28)
l2 = (15, 15, 25)
assert weak_compare(l1, l2, []) == 1
l1 = (12, 15, 20)
l2 = (15, 15, 25)
assert weak_compare(l1, l2, []) == 2

<a id="get_row"></a>
### get_row

`get_row` gets a row of values that represent a player's payoff **Used by**: [player1](#player1)

* **game** List[List[Tuple]]: the game world representing payoffs for each player and each strategy
* **index** int: the row desired
* **player** int: the player desired

**return**: List[int]: the row extracted from the game world

In [7]:
def get_row(game, index, player) -> List[int]:
    row = []
    for i, value in enumerate(game[0]):
        row.append(game[index][i][player])
    return row

In [8]:
test_game = [
    [(1, 2), (3, 4), (13, 14)],
    [(5, 6), (7, 8), (15, 16)],
    [(9, 10), (11, 12), (17, 18)],
]
assert get_row(test_game, 2, 0) == [9, 11, 17]
assert get_row(test_game, 2, 1) == [10, 12, 18]
assert get_row(test_game, 0, 0) == [1, 3, 13]


<a id="get_col"></a>
### get_col

`get_col` gets a column of values that represent a player's payoff **Used by**: [player2](#player2)

* **game** List[List[Tuple]]: the game world representing payoffs for each player and each strategy
* **index** int: the column desired
* **player** int: the player desired

**return**: List[int]: the row extracted from the game world

In [9]:
def get_col(game, index, player) -> List[int]:
    row = []
    for i, value in enumerate(game):
        row.append(game[i][index][player])
    return row

In [10]:
test_game = [
    [(1, 2), (3, 4), (13, 14)],
    [(5, 6), (7, 8), (15, 16)],
    [(9, 10), (11, 12), (17, 18)],
]
assert get_col(test_game, 2, 0) == [13, 15, 17]
assert get_col(test_game, 2, 1) == [14, 16, 18]
assert get_col(test_game, 0, 0) == [1, 5, 9]


<a id="get_equilibrium"></a>
### get_equilibrium

`get_equilibrium` finds the indexes of the equilibrium point if one exists by comparing the dominated rows and columns to the game rows and columns. If only 1 row and column haven't been dominated then that is the equilibrium point. If the game hasn't found an equilibrium point, then **None** is returned. **Used by**: [solve_game](#solve_game)

* **game** List[List[Tuple]]: the game world representing payoffs for each player and each strategy
* **d_row** List[int]: the list of dominated rows already identified
* **d_col** List[int]: the list of dominated columns already identified

**return**: Tuple: the index of a equilibrium point

In [11]:
def get_equilibrium(game, d_row, d_col) -> List:
    equilibrium_index = []
    if len(game) - len(d_row) != 1 or len(game[0]) - len(d_col) != 1:
        return
    for i, x in enumerate(game):
        if i not in d_row:
            equilibrium_index.append(i)

    for i, x in enumerate(game[0]):
        if i not in d_col:
            equilibrium_index.append(i)

    return equilibrium_index

In [12]:
test_game = [
    [(1, 2), (3, 4), (13, 14)],
    [(5, 6), (7, 8), (15, 16)],
    [(9, 10), (11, 12), (17, 18)],
]
test_rows = [0, 2]
test_cols = [0, 1]
assert get_equilibrium(test_game, test_rows, test_cols) == [1, 2]
test_rows = [0, 1]
test_cols = [0]
assert get_equilibrium(test_game, test_rows, test_cols) is None
test_rows = [0, 1, 2]
test_cols = [0, 1]
assert get_equilibrium(test_game, test_rows, test_cols) is None


<a id="player1"></a>
### player1

`player1` gets the rows to compare player 1 strategies and runs the strong or weak comparison.  **Uses**: [get_row](#get_row), [weak_compare](#weak_compare), [strong_compare](#strong_compare) **Used by**: [solve_game](#solve_game)

* **game** List[List[Tuple]]: the game world representing payoffs for each player and each strategy
* **col** int: the index of the desired column to compare against
* **d_row** List[int]: the list of dominated rows already identified
* **d_col** List[int]: the list of dominated columns already identified
* **weak** Bool: determines whether to use strong or weak domination

**return**: int: the index of a dominated row

In [13]:
def player1(game, row, d_row, d_col, weak: bool = False) -> int:
    player1_strategy = get_row(game, row, 0)
    if row not in d_row:
        for i in range(len(game)):
            if i not in d_row:
                player1_compare = get_row(game, i, 0)
                if weak:
                    dominate = weak_compare(player1_strategy, player1_compare, d_col)
                else:
                    dominate = strong_compare(player1_strategy, player1_compare, d_col)
                if dominate == 2:
                    return row
                if dominate == 1:
                    return i
    pass

In [14]:
test_game = [
    [(1, 2), (3, 4), (13, 14)],
    [(5, 6), (7, 8), (15, 16)],
    [(9, 10), (11, 12), (17, 18)],
]
test_rows = [0]
test_cols = [0, 1]
assert player1(test_game, 1, test_rows, test_cols) == 1
test_rows = []
test_cols = [0, 1]
assert player1(test_game, 0, test_rows, test_cols) == 0
test_rows = [0, 1]
test_cols = [0, 1]
assert player1(test_game, 2, test_rows, test_cols) is None


<a id="player2"></a>
### player2

`player2` gets the columns to compare player 2 strategies and runs the strong or weak comparison.  **Uses**: [get_col](#get_col), [weak_compare](#weak_compare), [strong_compare](#strong_compare) **Used by**: [solve_game](#solve_game)

* **game** List[List[Tuple]]: the game world representing payoffs for each player and each strategy
* **col** int: the index of the desired column to compare against
* **d_row** List[int]: the list of dominated rows already identified
* **d_col** List[int]: the list of dominated columns already identified
* **weak** Bool: determines whether to use strong or weak domination

**return**: int: the index of a dominated column

In [15]:
def player2(game, col, d_row, d_col, weak: bool = False) -> int:
    player2_strategy = get_col(game, col, 1)
    if col not in d_col:
        for i in range(len(game[0])):
            if i not in d_col:
                player2_compare = get_col(game, i, 1)
                if weak:
                    dominate = weak_compare(player2_strategy, player2_compare, d_row)
                else:
                    dominate = strong_compare(player2_strategy, player2_compare, d_row)
                if dominate == 2:
                    return col
                if dominate == 1:
                    return i
    pass

In [16]:
test_game = [
    [(1, 2), (3, 4), (13, 14)],
    [(5, 6), (7, 8), (15, 16)],
    [(9, 10), (11, 12), (17, 18)],
]
test_rows = [0]
test_cols = [0]
assert player2(test_game, 1, test_rows, test_cols) == 1
test_rows = []
test_cols = []
assert player2(test_game, 0, test_rows, test_cols) == 0
test_rows = [0, 1]
test_cols = [0, 1]
assert player2(test_game, 2, test_rows, test_cols) is None


<a id="solve_game"></a>
### solve_game

`solve_game` solves a game for equilibrium using either strong or weak dominating strategies **Uses**: [player1](#player1), [player2](#player2), [get_equilibrium](#get_equilibrium)

* **game** List[List[Tuple]]: the game world representing payoffs for each player and each strategy
* **weak** Bool: determines whether to use strong or weak domination

**return**: Tuple: the index of the equilibrium point

In [17]:
def solve_game(game: List[List[Tuple]], weak: bool = False) -> Tuple:
    dominated_row = []
    dominated_col = []
    row = 0
    col = 0
    while len(game) > row and len(game[0]) > col:
        player1_result = player1(game, row, dominated_row, dominated_col, weak)
        if player1_result is not None and player1_result not in dominated_row:
            dominated_row.append(player1_result)

        player2_result = player2(game, col, dominated_row, dominated_col, weak)
        if player2_result is not None and player2_result not in dominated_col:
            dominated_col.append(player2_result)

        row += 1
        col += 1

    e_index = get_equilibrium(game, dominated_row, dominated_col)
    return e_index

In [18]:
solve_game(prisoners_dilemma)

[0, 0]

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices.

For games that can be solved with *weak* SEDS, there may be more than one solution. You only need to return the first solution found. However, if you would like to return all solutions, you can implement `solve_game` as state space search.

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 14, 12 | 10, 10 | 14, 15 |
|1  | 20, 20 | 12, 14 | 28, 15 |
|2  | 15, 28 | 15, 14 | 25, 25 |

**Solution:**(1,0)

In [19]:
test_game_1 = [
    [(14, 12), (10, 10), (14, 15)],
    [(20, 20), (12, 14), (28, 15)],
    [(15, 28), (15, 14), (25, 25)],
]

solution = solve_game(test_game_1)


In [20]:
assert solution == [1, 0]

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 4, 2 | 0, 5 | 4, 5 |
|1  | 5, 0 | 2, 5 | 8, 5 |
|2  | 5, 8 | 5, 4 | 5, 5 |

**Solution:** (1,2)

In [21]:
test_game_2 = [
    [(4, 2), (0, 5), (4, 5)],
    [(5, 0), (2, 5), (8, 5)],
    [(5, 8), (5, 4), (5, 5)],
]

strong_solution = solve_game(test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

In [22]:
assert strong_solution is None
assert weak_solution == [1, 2]

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 4, 2 | 0, 5 | 4, 5 |
|1  | 5, 5 | 2, 5 | 8, 5 |
|2  | 5, 8 | 5, 4 | 5, 5 |

**Solution:** None

In [23]:
test_game_3 = [
    [(4, 2), (0, 5), (4, 5)],
    [(5, 5), (2, 5), (8, 5)],
    [(5, 8), (5, 4), (5, 5)],
]
strong_solution = solve_game(test_game_3)
weak_solution = solve_game(test_game_3, weak=True)

In [24]:
assert strong_solution is None
assert weak_solution is None